In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False


from IPython.display import display, HTML
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [3]:
import torch
import numpy as py
from torch import nn

from torch.nn import functional as F

In [91]:
x, y = torch.max(x_1, dim=3)
# print(x[0][0][0], x.shape)
x, y = torch.max(x, dim=3)
print(x[0][0], x.shape)

tensor([2.3973, 2.0273, 1.7461]) torch.Size([1, 8, 3])


In [25]:
import torch
a  = torch.empty(1, 64, 1).normal_(mean=0, std=0.02)

In [1]:
import torch

In [15]:
a = torch.randint(low=0, high=100, size=(2, 3, 4))
a

tensor([[[17,  5, 14, 64],
         [20, 79, 97, 70],
         [58, 68, 36, 90]],

        [[26, 94, 29,  3],
         [44, 20, 87, 36],
         [16, 39, 90, 96]]])

In [17]:
a.view(-1, 4).view(2, 3, 4)

tensor([[[17,  5, 14, 64],
         [20, 79, 97, 70],
         [58, 68, 36, 90]],

        [[26, 94, 29,  3],
         [44, 20, 87, 36],
         [16, 39, 90, 96]]])

In [24]:
b = torch.FloatTensor(a)

TypeError: expected Float (got Long)

In [40]:
a = torch.rand((10, ))
a

tensor([0.9458, 0.4751, 0.5290, 0.6003, 0.3570, 0.5608, 0.1706, 0.1137, 0.6722,
        0.4103])

In [41]:
a

tensor([0.9458, 0.4751, 0.5290, 0.6003, 0.3570, 0.5608, 0.1706, 0.1137, 0.6722,
        0.4103])

In [42]:
b = torch.nn.functional.dropout(a, p=0.5, inplace=False) 
a, b

(tensor([0.9458, 0.4751, 0.5290, 0.6003, 0.3570, 0.5608, 0.1706, 0.1137, 0.6722,
         0.4103]),
 tensor([1.8915, 0.9503, 0.0000, 0.0000, 0.7140, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]))